In [10]:
cd /data/alv/CTCF_degron/analysis/ChIPseq/ChIP4/input

/data/alv/CTCF_degron/analysis/ChIPseq/ChIP4/input


In [11]:
ls

DDX55-newpeaks
IAA-CTCF-narrowPeaks-sort-merge
IAA-CTCF-summits-sort-merge-200.bed
IAA-CTCF-summits-sort-merge-200-merge.bed
IAA-CTCF-summits-sort-merge.bed
IAA-DDX55-narrowPeaks-sort-merge
IAA-DDX55-summits-sort-merge-200.bed
IAA-DDX55-summits-sort-merge-200-merge.bed
IAA-DDX55-summits-sort-merge.bed
IAA-RAD21-narrowPeaks-sort-merge
IAA-RAD21-summits-sort-merge-200.bed
IAA-RAD21-summits-sort-merge-200-merge.bed
IAA-RAD21-summits-sort-merge.bed
IAA-TAF5L-narrowPeaks-sort-merge
IAA-TAF5L-summits-sort-merge-200.bed
IAA-TAF5L-summits-sort-merge-200-merge.bed
IAA-TAF5L-summits-sort-merge.bed
NT-CTCF-narrowPeaks-sort-merge
NT-CTCF-summits-sort-merge-200.bed
NT-CTCF-summits-sort-merge-200-merge.bed
NT-CTCF-summits-sort-merge.bed
NT-CTCF-summits-sort-merge-with-strength.bed
NT-DDX55-narrowPeaks-sort-merge
NT-DDX55-summits-sort-merge-200.bed
NT-DDX55-summits-sort-merge-200-merge.bed
NT-DDX55-summits-sort-merge.bed
NT-RAD21-narrowPeaks-sort-merge
NT-RAD21-summits-sort-merge-200.bed
NT-RAD21-sum

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from bioframe.tools import intersect
from bioframe import read_table

from itertools import combinations

In [13]:
import bioframe
chromsizes = bioframe.fetch_chromsizes('hg19')
chromosomes = list(chromsizes.index)

In [17]:
fnames = ["NT-CTCF-summits-sort-merge.bed"]

In [18]:
x = read_table(fnames[0],schema='bed')
x = x[x.chrom.isin(chromosomes)].reset_index(drop=True)

In [19]:
print(len(x))
x

37123


,chrom,start,end,name,score,strand
0,chr1,10434,10435,NaN,NaN,NaN
1,chr1,237753,237761,NaN,NaN,NaN
2,chr1,714190,714191,NaN,NaN,NaN
3,chr1,714278,714279,NaN,NaN,NaN
4,chr1,805287,805293,NaN,NaN,NaN
...,...,...,...,...,...,...
37118,chrY,13413213,13413214,NaN,NaN,NaN
37119,chrY,13420940,13420941,NaN,NaN,NaN
37120,chrY,13636710,13636715,NaN,NaN,NaN
37121,chrY,58968568,58968569,NaN,NaN,NaN


In [10]:
bedpes = []

gsep_min=200000
gsep_max=400000
df = x

for chrom in chromosomes[:-2]:
    # create pairwise combinations of indices of CTCF peaks for a given chrom:
    pairwise_index = pd.DataFrame(list(combinations(df.index[df['chrom']==chrom],2)))
    # left,right  of BEDPE ...
    left=df.iloc[pairwise_index[0]].reset_index(drop=True)[["chrom","start","end"]]
    right=df.iloc[pairwise_index[1]].reset_index(drop=True)[["chrom","start",'end']]
    # concat them ...
    cols = ["chrom1","start1","end1","chrom2","start2","end2"]
    bedpe = pd.concat([left,right],ignore_index=True,axis=1).rename(dict(enumerate(cols)),axis=1)
    # assert distances:
    genomic_seps = bedpe['start2']-bedpe['start1']
    #assert (genomic_seps>=0).all()
    # accumulate
    
    bedpes.append(bedpe[(genomic_seps>gsep_min)&(genomic_seps<gsep_max)])

bedpe_fin = pd.concat(bedpes, ignore_index=True)
bedpe_fin.to_csv("/data/alv/CTCF_degron/analysis/for-paper/draft4/figure2bis/dots/mao/G4-pairwise-200-400kb.bedpe")

In [11]:
fff = ["G4-pairwise-200-400kb.bedpe",
]

for f in fff:
    df = pd.read_csv("/data/alv/CTCF_degron/analysis/for-paper/draft4/figure2bis/dots/mao/"+f,index_col=0)
    df.to_csv("/data/alv/CTCF_degron/analysis/for-paper/draft4/figure2bis/dots/mao/"+f+".tsv",index=False,sep='\t')